In [5]:
from recbole.trainer import HyperTuning
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_seed
import os
from recbole.utils import get_model, get_trainer
from recbole.quick_start import objective_function


## Tuning BPR

In [19]:
hp = HyperTuning(objective_function=objective_function, algo='random', early_stop=10,
                max_evals=100, params_file='bpr.hyper', fixed_config_file_list=['foursquare_general.yaml'])


def objective_function(config_dict=None, config_file_list=None):
    k = 10

    config_dict = {
        #environment settings
        'model': 'BPR',
        'seed': 1234,
        'reproducibility': True,
        'data_path': os.getcwd(), 
        'topk': k,
        'use_gpu': True
    }

    config = Config(config_dict=config_dict, config_file_list=config_file_list)
    init_seed(config['seed'], config['reproducibility'])
    dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)
    model_name = config['model']
    model = get_model(model_name)(config, train_data._dataset).to(config['device'])
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
    test_result = trainer.evaluate(test_data)

    return {
        'model': model_name,
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result
    }

In [20]:
hp.run()
# export result to the file
hp.export_result(output_file='hyper_example.result')
# print best parameters
print('best params: ', hp.best_params)
# print best result
print('best result: ')
print(hp.params2result[hp.params2str(hp.best_params)])

running parameters:                                    
{'dynamic': False, 'embedding_size': 96, 'learning_rate': 0.0009442617993117138, 'mlp_hidden_size': '[64,64,64]', 'sample_num': 1, 'train_batch_size': 4096}
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

18 May 16:50    ERROR  job exception: 'model need to be specified in at least one of the these ways: [model variable, config file, config dict, command line] '


  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]


KeyError: 'model need to be specified in at least one of the these ways: [model variable, config file, config dict, command line] '